<a href="https://colab.research.google.com/github/gmauricio-toledo/NLP-LCC/blob/main/Notebooks/05-n_gramas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Modelos de lenguaje: $N$-gramas

Usaremos el corpus Reuters para entrenar un primer modelo de lenguaje. Este modelo lo usaremos para predicir la siguiente palabra en una secuencia de palabras dada.

El Corpus Reuters contiene 10,788 documentos de noticias con un total de 1.3 millones de palabras. Los documentos se han clasificado en 90 temas y agrupado en dos conjuntos, *train* y *test*.

* Corpus en NLTK: https://www.nltk.org/howto/corpus.html

In [ ]:
!gdown 1kvdbFQF9G1LTmcTVEPPy57nJRTuT2Kkt

## Modelo de 3-gramas

Exploremos un poco el corpus.

In [ ]:
import pandas as pd

df = pd.read_csv('wikipedia_batch_001_cleantext.csv')
df

In [ ]:
import re
import unicodedata

def limpiar_texto(texto):
    """
    Limpia un texto removiendo dígitos, puntuaciones, caracteres especiales,
    letras griegas, 'px', y otros elementos no deseados.

    Args:
        texto (str): Texto a limpiar

    Returns:
        str: Texto limpio
    """

    # Convertir a minúsculas
    texto = texto.lower()

    # Remover entidades HTML como &nbsp;
    texto = re.sub(r'&[a-z]+;', ' ', texto)

    # Remover 'px' (pixeles)
    texto = re.sub(r'\d*\.?\d+\s*px', ' ', texto)
    texto = re.sub(r'px', ' ', texto)

    # Remover letras griegas (alfabeto griego completo)
    # Rango Unicode para letras griegas: U+0370 a U+03FF
    texto = ''.join(char if not ('\u0370' <= char <= '\u03FF') else ' ' for char in texto)

    # Remover números (enteros y decimales)
    texto = re.sub(r'\d+\.?\d*', ' ', texto)

    # Remover símbolos matemáticos y científicos comunes
    texto = re.sub(r'[°±×÷√∞≈≠≤≥]', ' ', texto)

    # Remover puntuación y caracteres especiales
    # Mantener solo letras, espacios y algunos caracteres básicos
    texto = re.sub(r'[^\w\sáéíóúñüàèìòùâêîôûäëïöü]', ' ', texto)

    # Remover guiones bajos
    texto = re.sub(r'_+', ' ', texto)

    # Remover caracteres de control y formato
    texto = ''.join(char for char in texto if unicodedata.category(char)[0] != 'C')

    # Normalizar espacios múltiples a uno solo
    texto = re.sub(r'\s+', ' ', texto)

    return texto

In [ ]:
df['clean text'] = df['clean text'].apply(limpiar_texto)
df

In [ ]:
for text in df['clean text'].sample(2):
    print(text)

### Crear el modelo de lenguaje

In [ ]:
import nltk
from nltk import bigrams, trigrams
nltk.download('punkt_tab')

def crear_modelo_trigrama(texto):
    """
    Crea un modelo de trigramas desde un texto.

    texto: String con el texto de entrenamiento

    Returns:
        Modelo (diccionario de probabilidades)
    """
    # Tokenizar
    palabras = nltk.word_tokenize(texto.lower(), language='spanish')

    # Crear trigramas
    trigramas = []
    for i in range(len(palabras) - 2):
        trigramas.append((palabras[i], palabras[i+1], palabras[i+2]))

    # Alternativamente, crear trigramas con nltk
    # trigramas = list(trigrams(palabras))

    # Contar
    conteos = {}
    for w1, w2, w3 in trigramas:
        if (w1, w2) not in conteos:
            conteos[(w1, w2)] = {}
        if w3 not in conteos[(w1, w2)]:
            conteos[(w1, w2)][w3] = 0
        conteos[(w1, w2)][w3] += 1

    # Convertir a probabilidades
    modelo = {}
    for par, terceras in conteos.items():
        total = sum(terceras.values())
        modelo[par] = {palabra: cuenta/total for palabra, cuenta in terceras.items()}

    return modelo

Entrenamos con una parte del corpus

In [ ]:
corpus = "\n".join(df.sample(1000)['clean text'].values)

Agregamos un texto

In [ ]:
texto = "la universidad de sonora en hermosillo hará huelga"

corpus += "\n" + texto

In [ ]:
# Entrenar modelo
modelo = crear_modelo_trigrama(corpus)

print(f"\nModelo entrenado con {len(modelo)} pares de palabras únicos")

In [ ]:
modelo

In [ ]:
def predecir_siguiente(palabra1, palabra2, modelo):
    """
    Predice la siguiente palabra más probable.

    Args:
        palabra1: Primera palabra del contexto
        palabra2: Segunda palabra del contexto
        modelo: Diccionario con probabilidades

    Returns:
        La palabra más probable, o None si no hay predicción
    """
    par = (palabra1, palabra2)

    # Verificar si conocemos este par de palabras
    if par not in modelo:
        return None

    # Obtener la palabra con mayor probabilidad
    probabilidades = modelo[par]
    palabra_mas_probable = max(probabilidades, key=probabilidades.get)

    return palabra_mas_probable

def generar_texto(palabra_inicial1, palabra_inicial2, modelo, longitud=10):
    """
    Genera texto palabra por palabra usando el modelo.

    Args:
        palabra_inicial1: Primera palabra para empezar
        palabra_inicial2: Segunda palabra para empezar
        modelo: Modelo de n-gramas
        longitud: Cuántas palabras generar

    Returns:
        Texto generado como string
    """
    # Empezar con las dos palabras iniciales
    texto_generado = [palabra_inicial1, palabra_inicial2]

    for _ in range(longitud):
        # Usar las últimas dos palabras para predecir la siguiente
        w1 = texto_generado[-2]
        w2 = texto_generado[-1]

        siguiente = predecir_siguiente(w1, w2, modelo)

        # Si no hay predicción, terminar
        if siguiente is None:
            break

        texto_generado.append(siguiente)

    return ' '.join(texto_generado)

Generar texto con el modelo

In [ ]:
# Palabras de inicio:
inicio1, inicio2 = "las", "primeras"
# inicio1, inicio2 = "una", "investigación"
# inicio1, inicio2 = "de", "sonora"

longitud_maxima = 20

texto_generado = generar_texto(inicio1, inicio2, modelo, longitud=longitud_maxima)
print(f"Comenzando con '{inicio1} {inicio2}':\n\t{texto_generado}")

In [ ]:
# modelo[('las','primeras')]
modelo[('una','investigación')]

## ⭕ Estrategias de mejora

* Quitar o dejar signos de puntuación.
* Tokens de inicio y fin de texto
* Vocabulario cerrado $<$UNK$>$
* ¿Cómo serían las predicciones con un modelo de unigramas?
* Modificar parámetros de generación de texto

### Estrategias de Muestreo para Generación de Texto

Hasta ahora siempre elegimos la palabra **más probable**, pero esto hace que el texto sea repetitivo y predecible. Existen otras estrategias:

#### 1. **Top-k Sampling**
En lugar de elegir siempre la más probable, seleccionamos aleatoriamente entre las **k palabras más probables**.

**Ejemplo:** Si k=3 y tenemos:
- "café" (50%) ✅
- "té" (30%) ✅
- "chocolate" (15%) ✅
- "agua" (5%) ❌

Elegimos aleatoriamente solo entre café, té y chocolate (ignoramos "agua").

**Efecto:** Más variedad, pero controlada. Con k pequeño es conservador, con k grande es más creativo.

#### 2. **Top-p Sampling**
Seleccionamos entre las palabras cuyas probabilidades **suman p**.

**Ejemplo:** Con p=0.9:
- "café" (50%) ✅
- "té" (30%) ✅  
- "chocolate" (15%) ✅ $→$ Total: 95%
- "agua" (5%) ❌ (ya superamos 90%)

**Efecto:** Se adapta al contexto. Si hay una palabra muy probable (95%), solo considera esa. Si hay varias opciones, considera más palabras.


In [ ]:
!pip install -qq wordcloud

In [ ]:
from nltk import trigrams, bigrams

texto = " ".join(df['clean text'].sample(1000).values)
palabras = nltk.word_tokenize(texto.lower(), language='spanish')
trigramas = list(trigrams(palabras))
bigramas = list(bigrams(palabras))

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

tgs = [f"{w1}_{w2}_{w3}".lower() for w1,w2,w3 in trigramas]
bgs = [f"{w1}_{w2}".lower() for w1,w2 in bigramas]

wc = WordCloud(background_color='white')
# wc.generate(" ".join(tgs))
wc.generate(" ".join(bgs))
plt.imshow(wc)
plt.axis('off')
plt.show()

## Generalizaciones de $n$-gramas

### Skipgrams

Un $k$-skip-$n$-gram es una subsecuencia de longitud $n$ en la que los tokens aparecen a una distancia $k$ como máximo entre sí.


https://www.nltk.org/api/nltk.util.html

In [ ]:
from nltk.util import skipgrams
from string import punctuation

punctuations = list(punctuation)

text = "The rain in Spain falls mainly on the plain"

tokenized_text = [t for t in nltk.word_tokenize(text) if t not in punctuations]

skip_grams = list(skipgrams(tokenized_text, 2, 1))
print(skip_grams)